In [1]:
# by Joh Schöneberg and Cyna Shirazinejad 2019


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


### package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import skimage

### local new python segmentation functions
import os
import sys
pythonPackagePath = os.path.abspath('../src/')
sys.path.append(pythonPackagePath)
from peak_local_max_3d import peak_local_max_3d 
from gaussian_fitting import fit_multiple_gaussians
from extract_data import extract_data_from_filename
from gaussian_visualization import visualize_3D_gaussians


# Specify Input / Output Filenames

In [2]:
input_filename = '../test_data/cropped_488_pm50px_maxAmpl_0000.tif'
input_filename = '/Users/johannesschoeneberg/Dropbox/pylattice_testData/uncropped/S3P5_488_150mw_560_300mw_Objdz150nm_ch1_CAM1_stack0000_560nm_0000000msec_0090116101msecAbs_000x_000y_003z_0000t_decon.tif'

outputPath_tiff = ("../test_data/output_gaussians.tiff")
outputPath_csv = ("../test_data/output_gaussians.csv")

# View Raw Data


In [5]:
image_raw = extract_data_from_filename(input_filename)
view(single_fluorescent_view(image_raw))


Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

# Set Intensity Threshold

In [4]:
intensityThreshold = 10000
image_signal = image_raw>intensityThreshold
view(single_fluorescent_view(image_signal))

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

# Determine Local Maxima as Seeds for Gaussian Fitting

In [6]:
#~ 5s to run
maximas = peak_local_max_3d(image_raw,min_distance=10,threshold=10000)
print(len(maximas))
image_maxima = np.full(image_raw.shape,False)
for maxima in maximas:
    #print(maxima)
    image_maxima[maxima[0],maxima[1],maxima[2]] = 1000
view(single_fluorescent_view(image_maxima))

4788


Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

# Gaussian Fitting

In [7]:
sigmaExpected_x__pixels = 2
sigmaExpected_y__pixels = 2
sigmaExpected_z__pixels = 4

sigmas_guesses = []
for i in range(len(maximas)):
    sigmas_guesses.append([sigmaExpected_z__pixels,sigmaExpected_x__pixels,sigmaExpected_y__pixels])

In [8]:
#~100s to run
gaussians, gaussians_popt = fit_multiple_gaussians(image_raw,maximas,sigmas_guesses,5)

10%(479 of 4788)
20%(958 of 4788)
30%(1437 of 4788)
40%(1916 of 4788)
50%(2394 of 4788)
60%(2873 of 4788)
70%(3352 of 4788)
80%(3831 of 4788)
90%(4310 of 4788)
100%(4788 of 4788)


# Visualize the Fitted Gaussians

In [13]:
#takes ~15s
image_gaussians = visualize_3D_gaussians(image_raw,gaussians)
view(single_fluorescent_view(image_gaussians))

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

# Export Gaussians as tiff

In [14]:
skimage.external.tifffile.imsave(outputPath_tiff, image_gaussians.astype('uint16'))    
print(os.path.abspath(outputPath_tiff))

/Users/johannesschoeneberg/git/JohSchoeneberg/pyLattice_tutorials/test_data/output_gaussians.tiff


# Export Gaussians as Excel

In [15]:
accumulator = []
for gaussian in gaussians:

    if(gaussian!=-1):
        amplitude = 100*gaussian[0]
        
        #print(gaussian)
        mu_x     = int(gaussian[1][0])
        mu_y     = int(gaussian[1][1])
        mu_z     = int(gaussian[1][2])
        sigma_x  = int(gaussian[2][0])
        sigma_y  = int(gaussian[2][1])
        sigma_z  = int(gaussian[2][2])
        accumulator.append(np.array([amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z]))
accumulator = np.array(accumulator)
df = pd.DataFrame()
df['amplitude'] = accumulator[:,0]
df['mu_x'] = accumulator[:,1]
df['mu_y'] = accumulator[:,2]
df['my_z'] = accumulator[:,3]
df['sigma_x'] = accumulator[:,4]
df['sigma_y'] = accumulator[:,5]
df['sigma_z'] = accumulator[:,6]
df.to_csv(outputPath_csv)
print(os.path.abspath(outputPath_csv))

/Users/johannesschoeneberg/git/JohSchoeneberg/pyLattice_tutorials/test_data/output_gaussians.csv
